# ibm_db.execute()

## Purpose:

Execute an SQL statement that has been prepared by the __ibm_db.prepare()__ API.

## Syntax:

`bool ibm_db.execute( IBM_DBStatement `*`preparedStmt`*` [, tuple `*`paramValues`*`] )`

## Parameters:

* __*preparedStmt*__ &nbsp; &nbsp;: A prepared SQL statement returned from the __ibm_db.prepare()__ API.   
* __*paramValues*__ &nbsp; &nbsp; : A tuple that contains values for each parameter marker used in the prepared SQL statement specified in the __*preparedStmt*__ parameter.

## Return values:

* `True` &nbsp; &nbsp; &nbsp;: The prepared SQL statement specified in the __*preparedStmt*__ parameter executed successfully.   
* `False` &nbsp; &nbsp;: The prepared SQL statement specified in the __*preparedStmt*__ parameter could not be executed.

## Description:

The __ibm_db.execute()__ API is used to execute an SQL statement that has been prepared by the __ibm_db.prepare()__ API. If the SQL statement returns a result set, a single row can be retrieved and stored in a tuple or dictionary using the  __ibm_db.fetch_tuple()__ (tuple), __ibm_db.fetch_assoc()__ (dictionary), or __ibm_db.fetch_both()__ (tuple *and* dictionary) APIs. Alternately, the __ibm_db.fetch_row()__ API can be used to move the result set pointer to each row in the result set produced and the __ibm_db.result()__ API can be used to fetch a column from the current row. <p>
    
Refer to the notebook for __ibm_db.prepare()__ (i.e., the file *ibm_db-prepare.ipynb*) for a brief discussion of the advantages of using __ibm_db.prepare()__ and __ibm_db.execute()__, as opposed to __ibm_db.exec_immediate()__ to perform SQL operations against a Db2 database.

## Example:

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-execute.py                                                                 #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.execute() API.       #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.prepare()                                                             #
#                 ibm_db.fetch_tuple()                                                         #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The Db2ConnectionMgr Class Definition, Attributes, And Methods That Have Been Defined #
# In The File Named "ibm_db_tools.py"; This Class Contains The Programming Logic Needed To     #
# Establish And Terminate A Connection To A Db2 Server Or Database                             #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import Db2ConnectionMgr

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"
userID = "db2inst1"
passWord = "Passw0rd"
dbConnection = None
preparedStmt = False
returnCode = False
dataRecord = False

# Create An Instance Of The Db2ConnectionMgr Class And Use It To Connect To A Db2 Database
conn = Db2ConnectionMgr('DB', dbName, '', '', userID, passWord)
conn.openConnection()
if conn.returnCode is True:
    dbConnection = conn.connectionID
else:
    conn.closeConnection()
    exit(-1)

# Define The SQL Statement That Is To Be Executed
sqlStatement = "SELECT projno, projname FROM project WHERE prstaff < 2"

# Prepare The SQL Statement Just Defined
print("Preparing the SQL statement \"" + sqlStatement + "\" ... ", end="")
try:
    preparedStmt = ibm_db.prepare(dbConnection, sqlStatement)
except Exception:
    pass

# If The SQL Statement Could Not Be Prepared By Db2, Display An Error Message And Exit
if preparedStmt is False:
    print("\nERROR: Unable to prepare the SQL statement specified.")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Execute The SQL Statement Just Prepared
print("Executing the prepared SQL statement ... ", end="")
try:
    returnCode = ibm_db.execute(preparedStmt)
except Exception:
    pass

# If The SQL Statement Could Not Be Executed, Display An Error Message And Exit 
if returnCode is False:
    print("\nERROR: Unable to execute the SQL statement specified.")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Display A Report Header
print("Query results:\n")
print("PROJNO  PROJNAME")
print("______  _____________________")

# As Long As There Are Records In The Result Set Produced, ...
noData = False
while noData is False:

    # Retrieve A Record And Store It In A Python Tuple
    try:
        dataRecord = ibm_db.fetch_tuple(preparedStmt)
    except:
        pass

    # If The Data Could Not Be Retrieved Or There Was No Data To Retrieve, Set The
    # "No Data" Flag And Exit The Loop 
    if dataRecord is False:
        noData = True

    # Otherwise, Display The Data Retrieved
    else:
        print("{:<6}  {:24}" .format(dataRecord[0], dataRecord[1])) 

# Add A Blank Line To The End Of The Report
print()

# Close The Database Connection That Was Opened Earlier
conn.closeConnection()

# Return Control To The Operating System
exit()


Connecting to the SAMPLE database ... Done!

Preparing the SQL statement "SELECT projno, projname FROM project WHERE prstaff < 2" ... Done!

Executing the prepared SQL statement ... Done!

Query results:

PROJNO  PROJNAME
______  _____________________
AD3112  PERSONNEL PROGRAMMING   
IF2000  USER EDUCATION          
OP2011  SCP SYSTEMS SUPPORT     
OP2012  APPLICATIONS SUPPORT    
OP2013  DB/DC SUPPORT           
PL2100  WELD LINE PLANNING      

Disconnecting from the SAMPLE database ... Done!

